In [1]:
#Load initial libraries
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
#Get environment variables
pg_url2 = os.environ.get("PG_URL")
host2 = os.environ.get("DB_HOST")
database2 = os.environ.get("DB_NAME")
user2 = os.environ.get("DB_USER")
password2 = os.environ.get("DB_PASSWORD")

In [3]:
#connect to database
#for basic query
import psycopg2
import psycopg2.extras as extras
#for dataframe extract and load
from sqlalchemy import create_engine
engine = create_engine(pg_url2)

In [4]:
#Define a function to connect to the database. This was normally done with a one liner, and the example used a dict. 
#I ignored using the dict
def connect(host2, database2, user2, password2):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host=host2, database=database2, user=user2, password=password2)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

conn = connect(host2, database2, user2, password2)

Connecting to the PostgreSQL database...
Connection successful


In [5]:
#Get all of the symbols and ids needed
cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cursor.execute("""
    SELECT * FROM tickers WHERE id IN(SELECT DISTINCT holding_id from index_holdings)
""")
rows = cursor.fetchall()
rows[:4]

[[3, 'FOLD', 'AMICUS THERAPEUTICS INC', 'BBG000PLF6N0', 'Common Stock'],
 [5, 'HVT', 'HAVERTY FURNITURE', 'BBG000FVBPJ6', 'Common Stock'],
 [6, 'LTRN', 'LANTERN PHARMA INC', 'BBG00GM6LK17', 'Common Stock'],
 [12, 'EBET', 'ESPORTS TECHNOLOGIES INC', 'BBG00ZLGHH52', 'Common Stock']]

In [6]:
#query database and load price dataframe 
import pandas as pd
import talib
from talib_helpers.patterns import cdl_patterns


        
    

In [10]:
def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [15]:
for stock in rows:
    query_id = stock['id']
    #query_id = 16216
    query = """
        select * from stock_prices where stock_id = %s
    """ % (query_id)
    df = pd.read_sql_query(query, engine)
    for cdl in cdl_patterns:
        try:
            cdl_function = getattr(talib, cdl)
            result = cdl_function(df['open'], df['high'], df['low'], df['close'])
            df[cdl.lower()] = result
        except:
            pass
    df = df.drop(columns=['open', 'high', 'close', 'low', 'volume'])
    execute_values(conn, df, 'patterns')

Error: duplicate key value violates unique constraint "patterns_pkey"
DETAIL:  Key (stock_id, dt)=(3, 2007-05-31 00:00:00) already exists.

Error: duplicate key value violates unique constraint "patterns_pkey"
DETAIL:  Key (stock_id, dt)=(5, 1980-03-17 00:00:00) already exists.

Error: duplicate key value violates unique constraint "patterns_pkey"
DETAIL:  Key (stock_id, dt)=(6, 2020-06-11 00:00:00) already exists.

Error: duplicate key value violates unique constraint "patterns_pkey"
DETAIL:  Key (stock_id, dt)=(12, 2021-04-15 00:00:00) already exists.

Error: duplicate key value violates unique constraint "patterns_pkey"
DETAIL:  Key (stock_id, dt)=(26, 2017-11-02 00:00:00) already exists.

Error: duplicate key value violates unique constraint "patterns_pkey"
DETAIL:  Key (stock_id, dt)=(28, 2014-11-20 00:00:00) already exists.

Error: duplicate key value violates unique constraint "patterns_pkey"
DETAIL:  Key (stock_id, dt)=(39, 2005-07-22 00:00:00) already exists.

Error: duplicate

execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_va

execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_va

execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_va

execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_va

...

execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_va

execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_va

execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_va

execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_va

execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_va